# EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns



ModuleNotFoundError: No module named 'geopandas'

In [ ]:
alaska_data = pd.read_csv("../datasets/data_cleaned/alaska_single_engine_clean.csv")

1.Geographical Distribution: Plot the locations of the events on a map.

2.Time Series Analysis: Analyze the number of incidents over time (e.g., by year, month, or day).

3.Injury Analysis: Visualize the number of injuries or fatalities per event.

4.Aircraft Types: Show the distribution of events by aircraft make and model.

5.Correlation Heatmap: Examine the correlations between numerical variables.

6.Event Type Distribution: A bar chart of the different event types.

1.Geographical Distribution: Ploting the locations of the events on a map.

In [ ]:


alaska_data['latitude'] = pd.to_numeric(alaska_data['latitude'], errors='coerce') # Ensure correct type (float)
alaska_data['longitude'] = pd.to_numeric(alaska_data['longitude'], errors='coerce')

geo_data = alaska_data.dropna(subset=['latitude', 'longitude'])#

#GeoDataFrame
gdf = gpd.GeoDataFrame(
    geo_data, geometry=gpd.points_from_xy(geo_data.longitude, geo_data.latitude))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
alaska_map = world[(world.name == "United States of America") & (world.continent == "North America")]

fig, ax = plt.subplots(figsize=(15, 15))
alaska_map.plot(ax=ax, color='#78A7AF', edgecolor='black')
gdf.plot(ax=ax, markersize=20, color='#153C4B', marker='o', label='Incidents')
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
ax.set_title('Geographical Distribution of Aviation Incidents in Alaska')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.legend()
plt.show()


In [ ]:
#Zoom In

BBox = (-172.0, -130.0, 51.0, 71.5)
fig, ax = plt.subplots(figsize=(10, 16))
alaska_map.plot(ax=ax, color='#99C6BB', edgecolor='black')
gdf.plot(ax=ax, markersize=20, color='#153C4B', marker='o', label='Incidents')

ax.set_xlim(BBox[0], BBox[1])
ax.set_ylim(BBox[2], BBox[3])
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
ax.set_title('Geographical Distribution of Aviation Events in Alaska')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.grid()
plt.legend()
plt.show()


2.Time Series Analysis: Analyze the number of incidents over time (e.g., by year, month, or day).

In [ ]:
# unique values 
year_info = (alaska_data['event_year'].unique())
month_info = (alaska_data['event_month'].unique())
day_info = (alaska_data['event_day'].unique())

year_info, month_info, day_info


In [ ]:
# event_date
alaska_data['event_date'] = pd.to_datetime({
    'year': alaska_data['event_year'],
    'month': alaska_data['event_month'],
    'day': alaska_data['event_day']
})

incidents_per_year = alaska_data.groupby(alaska_data['event_date'].dt.year).size()# Group by event date (year) and count the number of incidents



plt.figure(figsize=(14, 7))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
incidents_per_year.plot(kind='line',color='#31606B', marker='o')
plt.title('Number of Aviation Incidents in Alaska Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Events')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()


3.Injury Analysis: Visualize the number of injuries or fatalities per event.

In [ ]:
injuries_data = alaska_data[["fatal_injury_count", "serious_injury_count", "minor_injury_count"]]

plt.figure(figsize=(14, 7))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
injuries_data.sum().plot(kind='bar', color='#31606B', stacked=True)
plt.title('Total Number of Injuries by Severity for Aviation Events in Alaska')
plt.xlabel('Injury Severity')
plt.ylabel('Total Number of Injuries')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


4.Aircraft Types: Show the distribution of events by aircraft make and model.

       4.1 Top 10 most common makes and models involved in incidents

In [ ]:
filtered_data = alaska_data[alaska_data['aircraft_family'] != 'UNCOMMON FAMILY']

top_makes_models = filtered_data['aircraft_family'].value_counts().head(10)


In [ ]:

plt.figure(figsize=(14, 7))
top_makes_models.plot(kind='bar', color='#78A7AF')
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
plt.title('Top 10 Aircraft Makes and Models Involved in Events in Alaska')
plt.xlabel('Aircraft Make and Model')
plt.ylabel('Number of Events')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()


In [ ]:
filtered_data = alaska_data[alaska_data['aircraft_family'] != 'UNCOMMON FAMILY']
top_makes_models = filtered_data['aircraft_family'].value_counts().head(10)
filtered_data = alaska_data[alaska_data['aircraft_family'].isin(top_makes_models.index)]
injuries_data = filtered_data.groupby('aircraft_family')['fatal_injury_count'].mean().loc[top_makes_models.index].sort_values(ascending=False)

plt.figure(figsize=(14, 7))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
injuries_data.plot(kind='bar', color='#78A7AF')
plt.title('Average Fatal Injury Count for Top 10 Aircraft Families Involved in an event in Alaska')
plt.xlabel('Aircraft Family')
plt.ylabel('Average Fatal Injury Count')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


6.Event Type Distribution: A bar chart of the different event types. # needs total numbers of flights

In [ ]:
event_type_counts = alaska_data['event_type'].value_counts()

plt.figure(figsize=(10, 6))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
event_type_counts.plot(kind='bar', color='#78A7AF')
plt.title('Distribution of Event Types for Aviation Events in Alaska')
plt.xlabel('Event Type')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


In [ ]:
alaska_data['event_time_of_day'].unique()

In [ ]:
event_type_counts = alaska_data['event_time_of_day'].value_counts()

plt.figure(figsize=(10, 6))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
event_type_counts.plot(kind='bar', color='#78A7AF')
plt.title('Distribution of Event Types based on time of day in Alaska')
plt.xlabel('Time of Day')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
event_type_counts = alaska_data['event_time_of_day'].value_counts()

plt.figure(figsize=(10, 6))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
event_type_counts.plot(kind='bar', color='#78A7AF')
plt.title('Distribution of Event Types based on time of day in Alaska')
plt.xlabel('Time of Day')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
filtered_data = alaska_data[alaska_data['event_time_of_day'].notna()]
top_makes_models = filtered_data['event_time_of_day'].value_counts().head(10)
filtered_data = alaska_data[alaska_data['event_time_of_day'].isin(top_makes_models.index)]
injuries_data = filtered_data.groupby('event_time_of_day')['fatal_injury_count'].mean().loc[top_makes_models.index].sort_values(ascending=False)

plt.figure(figsize=(14, 7))
fig.patch.set_facecolor('#C4E2DB')
ax.set_facecolor('#C4E2DB')
injuries_data.plot(kind='bar', color='#78A7AF')
plt.title('Distribution of Event Types based on time of day in Alaska')
plt.xlabel('Aircraft Family')
plt.ylabel('Average Fatal Injury Count')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()